In [ ]:
# http://eforexcel.com/wp/downloads-18-sample-csv-files-data-sets-for-testing-sales/

In [ ]:
from piper import piper
from piper.verbs import *

In [ ]:
def set_categories(df):
    
    categories = ['Region', 'Country', 'Item Type', 'Sales Channel', 'Order Priority']

    for category in categories:   
        categories = counts(df, category).index.tolist()
        categories = pd.CategoricalDtype(categories)
        df[category] = df[category].astype(categories)
        memory(df)
        
    return df

In [ ]:
def format_dates(df):
    
    logger.info('formatting dates...')
    
    dates = ['Order Date', 'Ship Date']
    
    for date_ in dates:
        df[date_] = pd.to_datetime(df[date_], format='%m/%d/%Y')
        
    return df

In [ ]:
xl_file = 'inputs/1000000 Sales Records.csv'

In [ ]:
%%piper
df <- pd.read_csv(xl_file)
>> drop(columns=['Total Revenue', 'Total Cost', 'Total Profit'])
>> distinct() # remove duplicate rows
>> format_dates()
>> set_categories() 

In [ ]:
duration_limit = pd.Timedelta(50, unit='days')

In [ ]:
%%piper 
df
>> clean_columns()
>> assign(lead_time=lambda x: x['ship_date'] - x['order_date'])
>> move_column('order_id', 'after', 'order_priority')
>> move_column('lead_time', 'after', 'ship_date')
>> where("lead_time >= @duration_limit", global_dict=globals())
>> head(5)

In [ ]:
%piper df >> info()

In [ ]:
%%piper 
counts(df, 'Region', percent=True, reset_index=True) 
>> adorn(ignore_row_index=True)

In [ ]:
%%piper 
df 
>> counts('Sales Channel', reset_index=True)
>> adorn(ignore_row_index=True)

In [ ]:
%%piper
df 
>> group_by(['Region', 'Order Priority'])
>> summarise(TotalUnitsSold=('Units Sold', 'sum')).unstack()
>> flatten_cols(remove_prefix='TotalUnitsSold')
>> pd.DataFrame.reset_index()
>> adorn(axis='column', ignore_row_index=True)
>> order_by(['All'], ascending=False)
>> adorn(ignore_row_index=True)
>> set_columns(columns=['Region', 'C', 'H', 'M', 'L', 'All'])